# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
orders = pd.read_csv("../Datasets as CSV/Orders.csv", index_col=0)
orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [21]:
# Group by CustomerID and sum amount_spent
orders_customer = orders.groupby(["CustomerID","Country"], as_index=False).amount_spent.sum().sort_values("amount_spent", ascending=False)
orders_customer

,CustomerID,Country,amount_spent
1698,14646,Netherlands,280206.02
4210,18102,United Kingdom,259657.30
3737,17450,United Kingdom,194550.79
3017,16446,United Kingdom,168472.50
1888,14911,EIRE,143825.06
...,...,...,...
4107,17956,United Kingdom,12.75
3023,16454,United Kingdom,6.90
1802,14792,United Kingdom,6.20
3226,16738,United Kingdom,3.75


In [25]:
# See summary statistics for amount_spent
orders_customer.amount_spent.describe()

count      4347.000000
mean       2050.013320
std        8980.103578
min           0.000000
25%         307.430000
50%         671.960000
75%        1661.195000
max      280206.020000
Name: amount_spent, dtype: float64

In [32]:
# Find amount_spent that corresponds to the 95th and the 75th percentile. 

quantile_75 = orders_customer["amount_spent"].quantile(q=0.75)
quantile_95 = orders_customer["amount_spent"].quantile(q=0.95)

print(quantile_75)
print(quantile_95)

1661.1950000000002
5774.277999999997


In [34]:
# Create new column assigning "VIP" or "Preferred" for each customer, depending on the percentile.

    # Define function 
def client_type(row):
    if row["amount_spent"]>=quantile_95:
        return "VIP"
    elif quantile_75<=row["amount_spent"]<quantile_95:
        return "Preferred"
    else:
        return "Regular"
    
    # Apply function to df
orders_customer["client_type"] = orders_customer.apply(client_type, axis=1)
  
orders_customer

,CustomerID,Country,amount_spent,client_type
1698,14646,Netherlands,280206.02,VIP
4210,18102,United Kingdom,259657.30,VIP
3737,17450,United Kingdom,194550.79,VIP
3017,16446,United Kingdom,168472.50,VIP
1888,14911,EIRE,143825.06,VIP
...,...,...,...,...
4107,17956,United Kingdom,12.75,Regular
3023,16454,United Kingdom,6.90,Regular
1802,14792,United Kingdom,6.20,Regular
3226,16738,United Kingdom,3.75,Regular


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [145]:
# Grouping clients by type, and counting total client by country, for each type:

orders_country = orders_customer.groupby("client_type").Country.value_counts()
print(orders_country)
print("\nCountries by descending number of VIP members:")
print(orders_country["VIP"])


client_type  Country       
Preferred    United Kingdom    756
             Germany            28
             France             20
             Belgium            11
             Norway              6
                              ... 
VIP          Finland             1
             Netherlands         1
             Norway              1
             Singapore           1
             Sweden              1
Name: Country, Length: 75, dtype: int64

Countries by descending number of VIP members:
Country
United Kingdom     178
Germany             11
France               9
Switzerland          3
EIRE                 2
Japan                2
Portugal             2
Spain                2
Australia            1
Channel Islands      1
Cyprus               1
Denmark              1
Finland              1
Netherlands          1
Norway               1
Singapore            1
Sweden               1
Name: Country, dtype: int64


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [212]:
# Adding new column where "Good" is assigned for "VIP" or "Preferred" clients, and "Bad" for regular clients

orders_customer["client_type_combined"]=["Good" if x=="VIP" or x=="Preferred" 
                                         else "Bad" for x in orders_customer["client_type"]]
orders_customer


,CustomerID,Country,amount_spent,client_type,client_type_combined
1698,14646,Netherlands,280206.02,VIP,Good
4210,18102,United Kingdom,259657.30,VIP,Good
3737,17450,United Kingdom,194550.79,VIP,Good
3017,16446,United Kingdom,168472.50,VIP,Good
1888,14911,EIRE,143825.06,VIP,Good
...,...,...,...,...,...
4107,17956,United Kingdom,12.75,Regular,Bad
3023,16454,United Kingdom,6.90,Regular,Bad
1802,14792,United Kingdom,6.20,Regular,Bad
3226,16738,United Kingdom,3.75,Regular,Bad


In [176]:
# Grouping clients by type, and counting total client by country, for each type:


orders_country_combined = orders_customer.groupby("client_type_combined").Country.value_counts()
orders_country_combined["Good"]


Country
United Kingdom     934
Germany             39
France              29
Belgium             11
Switzerland          9
Norway               7
Portugal             7
Spain                7
Finland              5
Italy                5
Australia            4
Channel Islands      4
Japan                4
Cyprus               3
Denmark              3
EIRE                 3
Israel               2
Sweden               2
Austria              1
Canada               1
Greece               1
Iceland              1
Lebanon              1
Malta                1
Netherlands          1
Poland               1
Singapore            1
Name: Country, dtype: int64

In [210]:
# Group by country, type and type combined, and add count of customerID by each type
orders_country_combined = orders_customer.groupby(["Country","client_type_combined", "client_type"], as_index=False).CustomerID.count()
orders_country_combined

,Country,client_type_combined,client_type,CustomerID
0,Australia,Bad,Regular,5
1,Australia,Good,Preferred,3
2,Australia,Good,VIP,1
3,Austria,Bad,Regular,10
4,Austria,Good,Preferred,1
...,...,...,...,...
70,United Arab Emirates,Bad,Regular,2
71,United Kingdom,Bad,Regular,2987
72,United Kingdom,Good,Preferred,756
73,United Kingdom,Good,VIP,178
